In [ ]:
import pandas as pd
import psycopg2
import json
import subprocess

In [1]:
import main

Installing postgresql...
Creating user...admin
Giving admin a password...
Giving admin CREATEDB permissions...
Making admin a superuser...
no creds.json file
Creating database localhost...
Database localhost created.
Connecting to database...
Logging in as admin
Database created raw_data........
Connecting to database...
Logging in as admin
database "raw_data" already exists

Connecting to database...
Logging in as admin
Data Downloaded
Looking for schema: imports
Copying table imports.data from /home/jupyter/data_engineering_project/data/data.csv...
relation "imports.data" does not exist

Table imports.data created.
Now... copying table imports.data from /home/jupyter/data_engineering_project/data/data.csv...
Table imports.data loaded.
Data loaded
Connecting to database...
Logging in as admin
Creating Dedupe Schema...
Creating dedupe table...
Dropping old table...
Renaming new table...
Renaming new table with new schema...
Dropping cleaning schema to clean up...
imports.data has been 

In [14]:
file=json.load(open('creds.json'))

conn = psycopg2.connect(database='data_mart',\
                        user=file['username'],\
                        password=file['password'],\
                        host='127.0.0.1',\
                        port= '5432')

cursor = conn.cursor()

In [15]:
#Select all table names in data_mart
sql = """
SELECT table_schema
        , table_name 
FROM 
    information_schema.tables
WHERE 
    table_schema NOT IN ('pg_catalog', 'information_schema')"""
cursor.execute(sql)
cursor.fetchall()
#{t[0]:t[1] for t in cursor.fetchall()}

[('loads', 'tmp_import'),
 ('customer_data', 'customers'),
 ('customer_data', 'address_history'),
 ('order_data', 'order_details'),
 ('order_data', 'orders')]

In [10]:
#cursor.close()
#conn.close()

In [16]:
#read a table from a postgres database using pandas
df = pd.read_sql('select * from loads.tmp_import', con=conn)

/var/tmp/ipykernel_39462/2673272983.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from loads.tmp_import', con=conn)


In [17]:
df.columns

Index(['order_id', 'first_name', 'last_name', 'date_of_birth', 'phone',
       'email', 'street', 'city', 'postal_code', 'state', 'order_date',
       'payment_type', 'product_name', 'price', 'quantity', 'order_total',
       'customer_id', 'address_id'],
      dtype='object')

In [18]:
#count the number of records in a df, grouped by first_name last_name

df[['first_name', 'last_name','date_of_birth',"street","city","state","postal_code"]]\
.groupby(['first_name', 'last_name','date_of_birth']).value_counts()

first_name  last_name  date_of_birth  street                      city           state  postal_code
Albina      Glick      1979-01-10     4 Ralph Ct                  Dunellen       NJ     8812            3
Allene      Iturbide   1979-01-02     1 Central Ave               Stevens Point  WI     54481           4
Amber       Monarrez   1979-02-02     55262 N French Rd           Indianapolis   IN     46240           5
Arlette     Honeywell  1979-02-28     30553 Washington Rd         Plainfield     NJ     7062            1
Art         Venere     1974-01-10     8 W Cerritos Ave #54        Bridgeport     NJ     8014            4
Bernardo    Figeroa    1979-01-06     386 9th Ave N               Conroe         TX     77301           1
Bette       Nicka      1978-12-28     6 S 33rd St                 Aston          PA     19014           3
Blondell    Pugh       1979-02-06     201 Hawk Ct                 Providence     RI     2904            2
Brock       Bolognia   1979-01-27     422 E 21st St 

In [19]:
df[['first_name', 'last_name','date_of_birth',"street","city","state","postal_code",'customer_id','address_id']]\
.groupby(['first_name', 'last_name','date_of_birth'])['customer_id'].nunique()
#.agg({'first_name': 'count', 'last_name': 'count', 'date_of_birth': 'count', 'street': 'count', 'city': 'count', 'state': 'count', 'postal_code': 'count'})

first_name  last_name  date_of_birth
Albina      Glick      1979-01-10       1
Allene      Iturbide   1979-01-02       1
Amber       Monarrez   1979-02-02       1
Arlette     Honeywell  1979-02-28       1
Art         Venere     1974-01-10       1
Bernardo    Figeroa    1979-01-06       1
Bette       Nicka      1978-12-28       1
Blondell    Pugh       1979-02-06       1
Brock       Bolognia   1979-01-27       1
Cammy       Albares    1978-09-22       1
Carma       Vanheusen  1979-03-13       1
Carmelina   Lindall    1979-02-09       1
Cecily      Hollack    1979-02-08       1
Cory        Gibes      1979-03-09       1
Delisa      Crupi      1979-02-23       1
Dyan        Oldroyd    1979-01-18       1
Elza        Lipke      1979-02-25       1
Emerson     Bowley     1979-01-25       1
Erick       Ferencz    1979-01-21       1
Ezekiel     Chui       1979-01-04       1
Gladys      Rim        1978-12-25       1
Graciela    Ruta       1978-03-21       1
Ilene       Eroman     1979-02-14      

In [20]:
df[['first_name', 'last_name','date_of_birth',"street","city","state","postal_code",'customer_id','address_id']]\
.groupby(['first_name', 'last_name','date_of_birth'])['address_id'].nunique()

first_name  last_name  date_of_birth
Albina      Glick      1979-01-10       1
Allene      Iturbide   1979-01-02       1
Amber       Monarrez   1979-02-02       1
Arlette     Honeywell  1979-02-28       1
Art         Venere     1974-01-10       1
Bernardo    Figeroa    1979-01-06       1
Bette       Nicka      1978-12-28       1
Blondell    Pugh       1979-02-06       1
Brock       Bolognia   1979-01-27       1
Cammy       Albares    1978-09-22       1
Carma       Vanheusen  1979-03-13       1
Carmelina   Lindall    1979-02-09       1
Cecily      Hollack    1979-02-08       1
Cory        Gibes      1979-03-09       1
Delisa      Crupi      1979-02-23       1
Dyan        Oldroyd    1979-01-18       1
Elza        Lipke      1979-02-25       1
Emerson     Bowley     1979-01-25       1
Erick       Ferencz    1979-01-21       1
Ezekiel     Chui       1979-01-04       1
Gladys      Rim        1978-12-25       2
Graciela    Ruta       1978-03-21       1
Ilene       Eroman     1979-02-14      

In [21]:
[('loads', 'tmp_import'),
 ('customer_data', 'customers'),
 ('customer_data', 'address_history'),
 ('order_data', 'order_details'),
 ('order_data', 'orders')]

[('loads', 'tmp_import'),
 ('customer_data', 'customers'),
 ('customer_data', 'address_history'),
 ('order_data', 'order_details'),
 ('order_data', 'orders')]

In [27]:
pd.read_sql('select * from customer_data.customers', con=conn).head(10)

/var/tmp/ipykernel_39462/2806696790.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from customer_data.customers', con=conn).head(10)


,customer_id,first_name,last_name,date_of_birth,phone,email
0,\xb792e6327a47271c310f58031374a3512ec650b689e2...,Malinda,Hochard,1979-03-14,317-722-5066,malinda.hochard@yahoo.com
1,\x173340a7c74ad18370a20dcbd486b598ff7ae6b0d662...,Albina,Glick,1979-01-10,732-924-7882,albina@glick.com
2,\xa7435a65a7e03a7f23533edf8375daf8f667c93090b3...,Allene,Iturbide,1979-01-02,715-662-6764,allene_iturbide@cox.net
3,\xc6d188bf1cc6c8c60ff64157303cc8985658f399860d...,Cory,Gibes,1979-03-09,626-572-1096,cory.gibes@gmail.com
4,\x62a2a8e32e53edb4800107b7bab342b4df8ef3ea0536...,Ilene,Eroman,1979-02-14,410-914-9018,ilene.eroman@hotmail.com
5,\x670b767168e36ff8fc8175f5aa715eda4ea94cdf02b7...,Vallie,Mondella,1979-02-15,208-862-5339,vmondella@mondella.com
6,\x7ba38823694b2ad8022642df27d970f814e575365a38...,Cammy,Albares,1978-09-22,956-537-6195,calbares@gmail.com
7,\xf819b452b787e756f9d51c382f82128c26a4b2d14ea7...,Dyan,Oldroyd,1979-01-18,913-413-4604,doldroyd@aol.com
8,\x7883b284e12c57017be0b75110d420e7a9ef460ca4b6...,Emerson,Bowley,1979-01-25,608-336-7444,emerson.bowley@bowley.org
9,\x180bc838a001fb45074e90a0647bed47178dbe1380df...,Gladys,Rim,1978-12-25,414-661-9598,gladys.rim@rim.org


In [28]:
pd.read_sql('select * from customer_data.address_history', con=conn).sort_values('customer_id').head(10)

/var/tmp/ipykernel_39462/2668644802.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from customer_data.address_history', con=conn).sort_values('customer_id').head(10)


,address_id,customer_id,street_number,city,postal_code,state,start_date,end_date
38,\x547bf2b5a532739b71e298ab104c4886a45f9931e567...,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,3777 E Richmond St #900,Akron,44302,OH,2015-02-26,2015-06-18
13,\x23e1c7eebe0fcb38bfb4c58280a3ed78cff4312dd70b...,\x04e39f45829b3256718e228e2c69e2ff10cc4102a6b8...,2371 Jerrold Ave,Kulpsville,19443,PA,2014-03-16,2016-06-18
53,\x6329c1db2f3070c153a185f48ad08c620a232795d5b4...,\x0516679ca04d7ec69dafa30c78fcbf53e65b0ef0dd33...,386 9th Ave N,Conroe,77301,TX,2013-07-09,2013-07-09
59,\x23cb652d80ad12872d7801feff5dc900ba9443033bb6...,\x0c74702d9a075245b45d32b0bb1b397b2d513768842c...,70 W Main St,Beachwood,44122,OH,2016-03-28,2016-03-28
52,\x4858de7b76f57ff3a27a556c5a650d2e457d6a6f563f...,\x0c74702d9a075245b45d32b0bb1b397b2d513768842c...,5161 Dorsett Rd,Homestead,33030,FL,2016-01-30,2016-01-30
8,\xae231da01a752ffedde5f5505a72a97f3241ca1f6695...,\x1563cbad451cdba0298fabbc2d76178b7a6ceebb29c5...,2664 Lewis Rd,Littleton,80126,CO,2015-07-23,2015-07-23
29,\xacb448f236fd0d380f998020d292956eedb2c50dd2f9...,\x173340a7c74ad18370a20dcbd486b598ff7ae6b0d662...,4 Ralph Ct,Dunellen,8812,NJ,2013-07-01,2013-07-01
42,\xe80ebcce1f4bc2b5fb922a414234340a9ff789cea399...,\x17b964b2e1a33c217ac4d6f71199c821ae04895427d4...,30553 Washington Rd,Plainfield,7062,NJ,2014-11-10,2014-11-10
56,\x6a419d1d8efcc51753c7494e436f29284d6063ba51b4...,\x180bc838a001fb45074e90a0647bed47178dbe1380df...,2239 Shawnee Mission Pky,Tullahoma,37388,TN,2013-05-06,2013-05-06
11,\xce38e1f08acbf647641a0f7e41804520a3b09450f536...,\x180bc838a001fb45074e90a0647bed47178dbe1380df...,322 New Horizon Blvd,Milwaukee,53207,WI,2017-02-02,2017-02-02


In [29]:
pd.read_sql('select * from order_data.order_details', con=conn).head(10)

/var/tmp/ipykernel_39462/3394181080.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from order_data.order_details', con=conn).head(10)


,product_name,order_id,price,quantity,comments
0,Ron-Flex,15,3.99,1,None
1,Mat Danin,36,94.99,3,None
2,Trans Zimnix,22,17.99,4,None
3,Trans Zimnix,92,10.99,1,None
4,Dentodox,31,5.99,1,None
5,Spanfind,31,72.99,3,None
6,Strongtam,41,50.99,4,None
7,Runhome,80,33.99,1,None
8,Stimstring,12,30.99,3,None
9,Redtom,33,90.99,2,None


In [30]:
pd.read_sql('select * from order_data.orders', con=conn).head(10)

/var/tmp/ipykernel_39462/566335479.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from order_data.orders', con=conn).head(10)


,order_id,customer_id,order_date,payment_method_id,comments
0,21,\x62a2a8e32e53edb4800107b7bab342b4df8ef3ea0536...,2017-01-02,ACH,None
1,50,\xd5169815bc622295666e2a920433ca2e889085daff2f...,2016-08-06,VISA,None
2,3,\xf8e87f42ce6114735a829d613995c57b9b5251acdc2f...,2016-11-29,VISA,None
3,29,\xb7e3b60b0014bf1a7b2718f2bfcf9c09981ff034e2cd...,2016-11-14,VISA,None
4,99,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,2015-06-18,VISA,None
5,7,\xb792e6327a47271c310f58031374a3512ec650b689e2...,2014-01-28,MASTER CARD,None
6,5,\xacbca24b7f82f7dd716511d7f19921575ed3338c1d7e...,2016-11-15,MASTER CARD,None
7,86,\x62a73b3a4c3f3fa5e07c126b211455c695409c88b080...,2015-08-06,MASTER CARD,None
8,94,\xd2ceae345ad9b173decfefeddb164a3c3fcdd7bf5b61...,2015-12-31,VISA,None
9,41,\x7ba38823694b2ad8022642df27d970f814e575365a38...,2015-06-15,ACH,None
